In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree

# Processing data

In [2]:
with open('SCS-Wood-Raw-Data.pkl', 'rb') as f:
    products = pickle.load(f)

#### Titles

In [3]:
with open('SCS-links_titles.pkl', 'rb') as f:
    titles_dict = pickle.load(f)

In [17]:
titles_df = pd.DataFrame([titles_dict]).T.reset_index().rename(columns={'index':'URL', 0:'Title'})

In [18]:
products = pd.merge(products,titles_df, on='URL', how='left')

#### Deduplication

In [20]:
products[~products.duplicated(subset=['Price','URL','Artikelnummer'])].shape
#'Stärke (in mm)-T','Breite (in mm)-T','Länge (in mm)-T',                         

(10417, 51)

In [21]:
products = products[~products.duplicated(keep='last')]
products.shape

(10418, 51)

#### Missed data

In [22]:
for col in products.columns:
    products[col] = products[col].apply(lambda x: np.nan if x=='' or x ==' '  else x)

#### Price/Delivery

In [101]:
test = products.copy()
test.shape

(10418, 51)

In [102]:
test['Price'] = test['Price'].str.replace(r' €|\.','')
test['Price'] = test['Price'].str.replace(r'\,','.').astype(float)

#### Drop missed columns

In [103]:
missed_cols = pd.DataFrame((test.isna().sum()/test.shape[0]*100), columns=['Missed'])
cols_del = missed_cols[missed_cols['Missed']>99].index.to_list()
len(cols_del)

24

In [104]:
test = test.drop(columns=cols_del)
test.shape

(10418, 27)

### Dimentions

In [105]:
for col in test.columns[:3]:
    test[col] = test[col].astype(str).str.replace(r' +',' ').apply(lambda x: np.nan if x=='' or x ==' '  else x)

In [106]:
test['Stärke (in mm)-T'] = test['Stärke (in mm)-T'].fillna(test['Höhe (in mm)-T']).fillna(test['Stärke (in mm)'])

In [107]:
test.isna().sum()/test.shape[0]*100

Stärke (in mm)-T        0.000000
Breite (in mm)-T        0.201574
Länge (in mm)-T         0.000000
Price                   0.000000
Delivery                0.000000
Description             0.000000
Artikelnummer           0.000000
Holzart                 0.662315
Holzfeuchte            84.526781
Oberfläche             84.018046
Profil                 86.484930
Qualität               84.584373
Gewicht ca. (kg/m3)    83.000576
Länge (in mm)          79.180265
Breite (in mm)          0.000000
Stärke (in mm)         83.250144
Lieferzeit              1.986946
URL                     0.000000
Est Quantity            0.000000
Actual Quantity         0.000000
Kanten                 98.819351
Höhe (in mm)-T         16.749856
Höhe (in mm)           16.749856
Quantity Delta         79.180265
URL count              79.180265
URL count Delta        79.180265
Title                   0.000000
dtype: float64

# Combine all columns to mine text form

In [108]:
test['Combined'] = ''
for col in ['Title', 'Description', 'Holzart', 'Holzfeuchte','Oberfläche', 'Profil', 'Qualität',  'Kanten', ]:
    test['Combined'] = test['Combined'] + '; ' + test[col].astype(str)

In [109]:
test['Combined'] = test['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

###  Quality

In [110]:
quality = ['A-Sortierung', 'B-Sortierung', 'C-Sortierung', 'A/B', 'B/C','Kl. A', '1.Wahl Sortierung', 'Altholz']

In [111]:
test['Quality'] = ''
for i in test.index:
    test['Quality'].loc[i] = []
    for qual in quality:
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(test['Quality'].loc[i]):
            test['Quality'].loc[i].append(qual)
            
    try:
        test['Quality'].loc[i] = ', '.join(test['Quality'].loc[i])
    except:
        pass
test['Quality'] =  test['Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Quality'].value_counts()

A-Sortierung, B-Sortierung    1272
B/C                             21
A-Sortierung                    14
A/B                              3
Name: Quality, dtype: int64

### Sorting Class

In [112]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe tragfähigkeit', 'mittlere tragfähigkeit',
               'hohe tragfähigkeit']

In [113]:
test['Sorting Class'] = ''
for i in test.index:
    test['Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(test['Sorting Class'].loc[i]):
            test['Sorting Class'].loc[i].append(sort_class)
       
    try:
        test['Sorting Class'].loc[i] = ', '.join(test['Sorting Class'].loc[i])
    except:
        pass
test['Sorting Class'] =  test['Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Sorting Class'].value_counts()

hohe tragfähigkeit    4
Name: Sorting Class, dtype: int64

## Main parameters

### Wood Types

In [114]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [115]:
test['Meta Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0:
            test['Meta Wood'].loc[i] = wood

In [116]:
test['Meta Wood'] = test['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [117]:
test['Wood Type'] = test['Holzart'].fillna(test['Meta Wood'])#.fillna(test['Holzart-Dekor-T1']).fillna(test['Meta Wood'])

#### Cheking wrong typs and woods

In [118]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
              'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl']

In [119]:
# Percentage of wrong wood types
test[(test['Wood Type'].isin(wrong_wood))].shape[0]/test.shape[0]*100

0.0

In [120]:
test[(test['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

Series([], Name: Wood Type, dtype: int64)

In [121]:
test = test[(~test['Wood Type'].isin(wrong_wood))]
test.shape

(10418, 32)

In [122]:
test['Wood Type'] = test['Wood Type'].str.lower()

### Surface

In [123]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [124]:
test['Combined Surface Treat'] = ''
for i in test.index:
    test['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Combined'].loc[i].lower() and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    try:
        test['Combined Surface Treat'].loc[i] = ', '.join(test['Combined Surface Treat'].loc[i])
    except:
        pass

In [125]:
test['Combined Surface Treat'] = test['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [126]:
test['Surface Treatment'] = test['Oberfläche'].fillna(test['Combined Surface Treat'])

### Surface Quality

In [127]:
surf_quality = ['NSI', 'SI']

In [128]:
test['Surface Quality'] = ''
for i in test.index:
    test['Surface Quality'].loc[i] = []
    for surf_qual in surf_quality:
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(test['Surface Quality'].loc[i]):
            test['Surface Quality'].loc[i].append(surf_qual)
            
    try:
        test['Surface Quality'].loc[i] = ', '.join(test['Surface Quality'].loc[i])
    except:
        pass
test['Surface Quality'] =  test['Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )   

In [129]:
test[~test['Surface Quality'].isna()].shape

(670, 35)

In [130]:
test['Surface Quality'].value_counts()

NSI    670
Name: Surface Quality, dtype: int64

### Drying method

In [131]:
dry_method = ['technischer trocknung','technisch getrocknet', 'kammer getrocknet',
               'kammertrocken', 'AD (luftgetrocknet)',
               'KD (Künstlich getrocknet)']

In [132]:
test['Combined Drying Method'] = ''
for i in test.index:
    test['Combined Drying Method'].loc[i] = []
    for method in dry_method:
        if method.lower() in test['Combined'].loc[i].lower() and method.lower() not in ', '.join(test['Combined Drying Method'].loc[i]):
            test['Combined Drying Method'].loc[i].append(method)
        
    try:
        test['Combined Drying Method'].loc[i] = ', '.join(test['Combined Drying Method'].loc[i])
    except:
        pass

In [133]:
test['Combined Drying Method'] =  test['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [134]:
test['Drying Method'] = test['Holzfeuchte'].fillna(test['Combined Drying Method'])

### Keywords

In [135]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [136]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [138]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

### Updating Product types

In [139]:
test['Product Type'] = np.nan
test['Product Type'].loc[0] = ''

In [140]:
for i in test.index:
    test['Product Type'].loc[i] = []
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in test['Combined'].loc[i].lower():
            if key_word.lower() == 'brett' and 'Brettschichtholz' in test['Product Type'].loc[i]:
                continue
            
            else:
                test['Product Type'].loc[i].append(key_word)
    test['Product Type'].loc[i] = ', '.join(test['Product Type'].loc[i])

In [141]:
wrong_products = ['Befestigungssysteme', 'Zubehör']

In [142]:
test = test[~test['Product Type'].isin(wrong_products)]
test.shape

(10418, 38)

### Matching keywords

In [143]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [144]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined', 'Surface Quality']

In [145]:
test['Matched Keywords'] = np.nan
test['Other Keywords'] = np.nan
test = test.fillna('-777')
keywords = keywords.fillna('-999')

In [146]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
test = test.apply(get_keywords, axis=1)

CPU times: user 2min 46s, sys: 1.73 s, total: 2min 48s
Wall time: 3min 10s


In [147]:
test[['Matched Keywords','Other Keywords']].isna().sum()/test.shape[0]*100

Matched Keywords    0.39355
Other Keywords      0.39355
dtype: float64

### Cleaning NaNs



In [148]:
for col in test.columns:
    test[col] =  test[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [149]:
test[(test['Matched Keywords'].isna())].shape

(41, 40)

In [151]:
test[(test['Matched Keywords'].isna()) & 
          (test['Wood Type']=='Holz')].shape

(0, 40)

In [153]:
test[(~test['Matched Keywords'].isna()) #&
          ]['Product Type'].nunique()#value_counts()

0

### Pre-cleaning

In [154]:
for col in test.columns:
    test[col] = test[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [292]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'

In [155]:
sheets_clean = test.copy()

In [156]:
sheets_clean = test[~test['Matched Keywords'].isna()]
sheets_clean.shape

(10377, 40)

In [298]:
sheets_clean['Website Name'] = 'www.scs-holzshop.de'

In [299]:
ren_cols = {'Title':'Product Title',
            'Breite-T1-mm':'Width (mm)',
            'Länge-T1-mm':'Length (mm)', 
            'Stärke-T1-mm':'Thickness (mm)',
            'Price':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Delivery Yes/No','Quality', 'Surface Treatment','Drying Method', 'Surface Quality',
        'Sorting Class','SKU','Delivery Status' ]

sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [300]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]

In [301]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

16.64325842696629

In [302]:
d2g.upload(sheets_clean.fillna('-'),
           spreadsheet_key,
           'OBI Clean Data',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'OBI Clean Data' id:667286089>

# *